In [27]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [28]:
df = pd.read_csv('RawDataDoNotModify/BTCUSD_Candlestick_1_Hour_BID_01.01.2020-31.08.2024.csv')
## Remove rows with no volume
df = df[df['Volume']!=0]
df = df.tail(5000)
df.reset_index(drop=True, inplace=True)
## volume MA
df['volume_MA'] = df['Volume'].rolling(window=7).mean()

In [30]:
class SupportResistance():
    def __init__(self, df):
        self.df = df
        self.bsp = 10
        self.asp = 10
        self.price_range_pct = 1
        self.x_candles = 720
    
    def pivotid(self, index):
        """
        1 - support
        2 - resistance
        0 - Nothing
        """
        ## edge conditions
        if index - self.bsp < 0 or index + self.asp >= len(self.df):
            return 0
        pividlow = 1
        pividhigh = 1
        for i in range(index-self.bsp, index+self.asp+1):
            if self.df.Low[index] > self.df.Low[i]:
                pividlow = 0
            if self.df.High[index] < self.df.High[i]:
                pividhigh = 0
        if pividlow and pividhigh:
            return 3
        elif pividlow:
            return 1
        elif pividhigh:
            return 2
        else:
            return 0
        

    def pointpos(self, x):
        if x['pivot']==1:
            return x['Low']-1e-2
        elif x['pivot']==2:
            return x['High']+1e-2
        else:
            return np.nan

    
    def calculate_dynamic_support_resistance(self, df, percentage, x_candles):
        # Add new columns for support and resistance based on the 'pivot' column
        df['support'] = df.apply(lambda row: row['pointpos'] if row['pivot'] == 1 else None, axis=1)
        df['resistance'] = df.apply(lambda row: row['pointpos'] if row['pivot'] == 2 else None, axis=1)

        # Initialize lists for closest support and resistance
        closest_supports = []
        closest_resistances = []

        # Process each row to calculate supports and resistances dynamically
        for i in range(len(df)):
            # Get past data (rows prior to the current row)
            past_data = df.iloc[max(0, i-x_candles):i]

            if past_data.empty:
                # If no past data, append None
                closest_supports.append(None)
                closest_resistances.append(None)
                continue

            # Filter supports and resistances from past data
            past_supports = past_data['support'].dropna()
            past_resistances = past_data['resistance'].dropna()

            # Calculate price ranges dynamically based on past data Close prices
            price_range_support = df.loc[i, 'Close'] * (percentage / 100)
            price_range_resistance = df.loc[i, 'Close'] * (percentage / 100)

            # Aggregate supports and resistances within the dynamic price range
            def aggregate_points(points, price_range):
                aggregated_points = []
                while len(points) > 0:
                    current = points.iloc[0]
                    close_points = points[abs(points - current) <= price_range]
                    aggregated_points.append(max(close_points))
                    points = points[abs(points - current) > price_range]
                return aggregated_points

            avg_supports = aggregate_points(past_supports, price_range_support)
            avg_resistances = aggregate_points(past_resistances, price_range_resistance)

            # Find the closest aggregated support and resistance for the current row
            current_open = df.loc[i, 'Open']
            current_high = df.loc[i, 'High']

            closest_support = (
                min(avg_supports, key=lambda s: abs(s - current_open)) if avg_supports else None
            )
            closest_resistance = (
                min(avg_resistances, key=lambda r: abs(r - current_high)) if avg_resistances else None
            )

            closest_supports.append(closest_support)
            closest_resistances.append(closest_resistance)

        # Add closest support and resistance back to the DataFrame
        df['closest_support'] = closest_supports
        df['closest_resistance'] = closest_resistances

        return df

    
    def run(self):
        self.df['pivot'] = df.apply(lambda x: self.pivotid(x.name), axis=1)
        self.df['pointpos'] = df.apply(lambda row: self.pointpos(row), axis=1)
        self.df = self.calculate_dynamic_support_resistance(self.df, self.price_range_pct, self.x_candles)
        
obj = SupportResistance(df)
obj.run()
test_df = obj.df

In [ ]:
class ShortSupport():
    def __init__(self,df):
        self.df = df
        self.capital = 100_000
        self.activte_trade = None
        self.entry = 0
        self.sl = 0
        self.tp = 0
        self.sl_buffer = 0.15
        self.tp_buffer = 0.20
        self.total_trades = 0
    
    def plots_graph_with_custom_lines(self, dfpl, line1, line2, line3):
        # Prepare the candlestick chart
        fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                                             open=dfpl['Open'],
                                             high=dfpl['High'],
                                             low=dfpl['Low'],
                                             close=dfpl['Close'],
                                             increasing_line_color='green',
                                             decreasing_line_color='red')])

        # Add pivot points as markers
        fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                        marker=dict(size=5, color="MediumPurple"),
                        name="pivot")

        # Add unique closest resistance levels as horizontal lines
        unique_resistances = dfpl['closest_support'].dropna().unique()
        for resistance in unique_resistances:
            fig.add_shape(type="line",
                          x0=dfpl.index.min(), x1=dfpl.index.max(),
                          y0=resistance, y1=resistance,
                          line=dict(color="blue", width=1),
                          name=f"Resistance: {resistance}")

        # Add custom horizontal lines
        d = {line1:'white',line2:'red',line3:'green'}
        for key,value in d.items():
            fig.add_shape(type="line",
                          x0=dfpl.index.min(), x1=dfpl.index.max(),
                          y0=key, y1=key,
                          line=dict(color=value, width=1, dash="dash"),
                          name=f"Custom Line: {key}")

        # Update layout for aesthetics
        fig.update_layout(xaxis_rangeslider_visible=False)
        fig.update_xaxes(showgrid=False)
        fig.update_yaxes(showgrid=False)
        fig.update_layout(paper_bgcolor='black', plot_bgcolor='black')

        # Show the plot
        fig.show() 
        
    def strategy(self):
        for i in range(len(self.df)):
            row = self.df.iloc[i]
            if not row['closest_support'] or not row['closest_resistance'] :
                continue 
            ## To TRADE Condition
            if not self.activte_trade:
                if (row['High'] > row['closest_resistance'] 
                    and row['Close'] < row['closest_resistance'] 
                    and row['Close']<row['Open']
                   ):
                self.entry = row['Close']
                self.sl = row['High'] * (1+ (self.sl_buffer/100))
                self.tp = row['closest_support'] * (1+ (self.tp_buffer/100))
                if ( self.entry < self.tp 
                    or abs(self.entry - self.sl) > abs(self.entry - self.tp)
                    or row['Volume'] < row['volume_MA']
                   ):
                    continue
                self.activte_trade = 'Short'
                dfpl = self.df[i-5:i+5]
                self.plots_graph_with_custom_lines(dfpl,self.entry,self.sl,self.tp )
            ## TO EXIT Condition
            elif self.activte_trade == 'Short':
                if row['Low'] < self.tp:
                    profit = self.entry - self.tp
                    self.capital += profit
                    self.activte_trade = None
                elif row['High'] > self.sl:
                    profit = self.sl - self.entry
                    self.capital -= profit
                    self.activte_trade = None

obj = ShortSupport()
